연습문제
목표가 설정
아래 공식을 사용하여 목표가(Target)를 계산합니다.
목표가 = 당일 시가 + (전일 변동폭 * K)
변동폭 = 전일 고가 - 전일 저가
매수 조건
장중 고가가 목표가 이상일 경우 매수.
K값 수익률 분석
다양한 K값(예: 0.1 ~ 1.0)을 사용해 각 K값에 따른 전략 수익률을 계산하세요.
최적 K값 선택
가장 높은 누적 수익률(Cumulative Return)을 기록한 K값을 구하세요.





In [47]:
import pandas as pd 
from pykrx import stock # 9장에 사용법 나옴

In [48]:
df_kosdaq_150 = stock.get_market_ohlcv_by_date('20101001','20241230','252670')
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률
날짜,,,,,,
2016-09-22,9835,9845,9720,9815,7849515,NaN
2016-09-23,9755,9840,9755,9820,9775392,0.050942
2016-09-26,9840,9910,9735,9875,10065403,0.560081
2016-09-27,9975,10040,9705,9745,15822705,-1.316456
2016-09-28,9765,9820,9745,9805,9475753,0.615700


In [49]:
df_kosdaq_150['변동폭'] = df_kosdaq_150['고가'] - df_kosdaq_150['저가']
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭
날짜,,,,,,,
2016-09-22,9835,9845,9720,9815,7849515,NaN,125
2016-09-23,9755,9840,9755,9820,9775392,0.050942,85
2016-09-26,9840,9910,9735,9875,10065403,0.560081,175
2016-09-27,9975,10040,9705,9745,15822705,-1.316456,335
2016-09-28,9765,9820,9745,9805,9475753,0.615700,75


In [50]:
df_kosdaq_150['전일변동폭'] = df_kosdaq_150['변동폭'].shift(periods=1) #전일 변동폭이라서 
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭
날짜,,,,,,,,
2016-09-22,9835,9845,9720,9815,7849515,NaN,125,NaN
2016-09-23,9755,9840,9755,9820,9775392,0.050942,85,125.0
2016-09-26,9840,9910,9735,9875,10065403,0.560081,175,85.0
2016-09-27,9975,10040,9705,9745,15822705,-1.316456,335,175.0
2016-09-28,9765,9820,9745,9805,9475753,0.615700,75,335.0


In [51]:
K = 1/2
df_kosdaq_150['목표가'] = df_kosdaq_150['시가'] + (df_kosdaq_150['전일변동폭'] * K)
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭,목표가
날짜,,,,,,,,,
2016-09-22,9835,9845,9720,9815,7849515,NaN,125,NaN,NaN
2016-09-23,9755,9840,9755,9820,9775392,0.050942,85,125.0,9817.5
2016-09-26,9840,9910,9735,9875,10065403,0.560081,175,85.0,9882.5
2016-09-27,9975,10040,9705,9745,15822705,-1.316456,335,175.0,10062.5
2016-09-28,9765,9820,9745,9805,9475753,0.615700,75,335.0,9932.5


In [52]:
conditions = df_kosdaq_150['고가'] >= df_kosdaq_150['목표가']  # 매도 조건

In [53]:
import numpy as np
df_kosdaq_150['수익률'] = np.where(conditions, df_kosdaq_150['종가'] / df_kosdaq_150['목표가'], 1)

In [54]:
delta_days = df_kosdaq_150.index[-1] - df_kosdaq_150.index[0]
year = delta_days.days / 365

CAGR_dict = {}
for k in range(1,11):
    df_kosdaq_150['목표가'] = df_kosdaq_150['시가'] + (df_kosdaq_150['전일변동폭'] * (k/10))
    df_kosdaq_150['수익률'] = np.where(conditions, df_kosdaq_150['종가'] / df_kosdaq_150['목표가'], 1)
    df_kosdaq_150['누적수익률'] = df_kosdaq_150['수익률'].cumprod()
    CAGR = df_kosdaq_150['누적수익률'].iloc[-1] ** (1/year) -1
    CAGR_dict[f'{(k/10)}'] = CAGR * 100


CAGR_dict

{'0.1': 195.9587106845218,
 '0.2': 132.29406057808686,
 '0.3': 82.4506951573755,
 '0.4': 43.40052797809126,
 '0.5': 12.784968031460831,
 '0.6': -11.234455675907107,
 '0.7': -30.091837070603532,
 '0.8': -44.906645121976666,
 '0.9': -56.55337462095108,
 '1.0': -65.71563843036361}